In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [57]:
train =pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")

In [58]:
test =pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [59]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [60]:
!pip install scikit-learn

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [62]:
categorical_columns = train.select_dtypes(include=['object', 'category']).columns

# Print the list of categorical columns
print("Categorical Columns:")
print(categorical_columns)

Categorical Columns:
Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')


In [63]:
for column in train.columns:
    if train[column].dtype == 'float64' or train[column].dtype == 'int64':
        train.fillna({column: train[column].mean()}, inplace=True)
    elif train[column].dtype == 'object' or train[column].dtype.name == 'category':
        mode_value = train[column].mode()[0]
        train.fillna({column: mode_value}, inplace=True)


In [64]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder
for i in train.columns:
    if i in categorical_columns:
        train[i] = LabelEncoder().fit_transform(train[i])
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,65.0,8450,1,0,3,3,0,...,0,2,2,2,0,2,2008,8,4,208500
1,2,20,3,80.0,9600,1,0,3,3,0,...,0,2,2,2,0,5,2007,8,4,181500
2,3,60,3,68.0,11250,1,0,0,3,0,...,0,2,2,2,0,9,2008,8,4,223500
3,4,70,3,60.0,9550,1,0,0,3,0,...,0,2,2,2,0,2,2006,8,0,140000
4,5,60,3,84.0,14260,1,0,0,3,0,...,0,2,2,2,0,12,2008,8,4,250000


In [65]:
X = train.drop('SalePrice', axis=1)  # Features
y = train['SalePrice']  # Target variable

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [67]:
coefficients = model.coef_
intercept = model.intercept_

# Print the coefficients and the intercept
print("Coefficients (Weights):", coefficients)
print("Intercept:", intercept)


Coefficients (Weights): [-1.95615504e+00 -1.19773437e+02 -3.05097084e+03 -2.52887690e+02
  3.51917067e-01  2.18123623e+04 -3.13180520e+03 -1.13969963e+03
  2.43747945e+03 -4.88854918e+04 -1.97708425e+01  1.11520802e+04
  3.97040535e+02 -3.83348556e+02 -1.13425134e+04 -2.54916797e+03
 -1.05417661e+03  1.04562463e+04  5.26480397e+03  1.87732896e+02
  6.21684351e+01  2.20213630e+03  5.87597631e+03 -1.53285855e+03
  8.84130946e+02  3.26052585e+03  2.17503198e+01 -8.61887133e+03
  1.07008928e+02  6.89826431e+02 -9.88205080e+03  3.39993035e+03
 -3.48771988e+03 -3.49611617e+02  3.16740135e+00  1.36931852e+03
  5.72341881e+00 -4.39842371e+00  4.49239661e+00 -3.49766764e+02
 -1.78233182e+02  7.44391934e+02 -5.11955158e+02  2.47251568e+01
  2.10334511e+01 -2.44363563e+01  2.13222515e+01  7.41399094e+03
 -4.31352576e+03  1.75307927e+03 -9.48589753e+02 -2.01898590e+03
 -1.35592652e+04 -8.97974260e+03  4.08857873e+03  4.53087962e+03
  6.42576218e+03 -1.56310721e+03 -8.10677443e+02  2.88642088e+01
 

In [68]:
y_pred = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 1248056960.5160668
R^2 Score: 0.8372876293909596


In [69]:
from sklearn.preprocessing import LabelEncoder
for i in test.columns:
    if i in categorical_columns:
        test[i] = LabelEncoder().fit_transform(test[i])
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,2,80.0,11622,1,2,3,3,0,...,120,0,2,2,3,0,6,2010,8,4
1,1462,20,3,81.0,14267,1,2,0,3,0,...,0,0,2,4,0,12500,6,2010,8,4
2,1463,60,3,74.0,13830,1,2,0,3,0,...,0,0,2,2,3,0,3,2010,8,4
3,1464,60,3,78.0,9978,1,2,0,3,0,...,0,0,2,4,3,0,6,2010,8,4
4,1465,120,3,43.0,5005,1,2,0,1,0,...,144,0,2,4,3,0,1,2010,8,4


In [70]:
for column in test.columns:
    if test[column].dtype == 'float64' or test[column].dtype == 'int64':
        test.fillna({column: test[column].mean()}, inplace=True)
    elif test[column].dtype == 'object' or test[column].dtype.name == 'category':
        mode_value = test[column].mode()[0]
        test.fillna({column: mode_value}, inplace=True)


In [71]:
ans=model.predict(test)

In [72]:
ans

array([ 77115.31326471, 153660.36314065, 138399.14393064, ...,
       107421.32932663, 101909.04966211, 211967.17516482])

In [73]:
submission = pd.DataFrame({"Id": test.Id, "SalePrice": ans})
submission

,Id,SalePrice
0,1461,77115.313265
1,1462,153660.363141
2,1463,138399.143931
3,1464,155087.653673
4,1465,158204.581985
...,...,...
1454,2915,35715.085795
1455,2916,30978.098228
1456,2917,107421.329327
1457,2918,101909.049662


In [74]:
submission.to_csv('submission.csv', index = False)